In [1]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch==1.3.1

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-td9698la
  Created wheel for kobert: filename=kobert-0.1.1-py3-none-any.whl size=12824 sha256=a205918f64cedf2e4a9aaaaac4cbb43867e961850490653a5d9f6562ad5b4f53
  Stored in directory: /tmp/pip-ephem-wheel-cache-07kw6jn2/wheels/4b/9b/3f/583189713d96fda0f291ca1f228ab0e0c981641dcdaf7c7cdf
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


OSError: libcublas.so.10: cannot open shared object file: No such file or directory

In [ ]:
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [ ]:
dataset_train = nlp.data.TSVDataset("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/dacon_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/dacon_test.txt", field_indices=[1,2], num_discard_samples=1)
#dataset_submission = nlp.data.TSVDataset("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/dacon_submission.txt", field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5, shuffle=False)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

## submission data 

In [ ]:
dataset_submission = nlp.data.TSVDataset("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/dacon_submission.txt", field_indices=[1,2])

In [ ]:
data_submission = BERTDataset(dataset_submission, 0, 1, tok, max_len, True, False)

In [ ]:
submission_dataloader = torch.utils.data.DataLoader(data_submission, batch_size=1, num_workers=5, shuffle=False)

In [ ]:
score_dict ={}
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5



for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(submission_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    test_acc += calc_accuracy(out, label)
    
    score_dict[dataset_submission[batch_id][1]] = out.cpu().data.numpy()[0][1]
    
    #if batch_id%100 == 0:
    #    print(batch_id/len(dataset_submission)*100)
    """print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    print(token_ids)
    print(label)
    print(dataset_submission[batch_id][1])
    """

In [ ]:
sub_data.head()

In [ ]:
sub_data = pd.read_csv("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/dacon_submission.txt", sep="\t", header=None)
sub_data["score"] = sub_data[2].map(lambda x: score_dict[str(x)])
sub_data["sent_num"] = sub_data[2].map(lambda x: str(x)[:-3])


sub_data["rank"] = sub_data.groupby("sent_num")["score"].rank(method="min", ascending=False)

result_dict={}

sent_list = list(set(sub_data["sent_num"]))

for sent in sent_list:
    tmp = sub_data[sub_data["sent_num"]==sent]
    
    answer = tmp[tmp["rank"]==1][1].item() + "\n" + tmp[tmp["rank"]==2][1].item() + "\n" + tmp[tmp["rank"]==3][1].item()
    
    result_dict[sent] = answer

In [ ]:
sub_data.head()

In [ ]:
sub_fi = pd.read_csv("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/extractive_sample_submission_v2.csv")
sub_fi["summary"] = sub_fi["id"].map(lambda x: result_dict[str(x)])
sub_fi.to_csv("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/ext_sub_order.csv", index=False)

### 순서대로 변경 

In [ ]:
sub_data["imp_sentence"] = sub_data["rank"].map(lambda x: 1 if x <=3 else 0)

In [ ]:
for sent in sent_list:
    tmp = sub_data[sub_data["sent_num"]==sent]
    
    answer = "\n".join(list(tmp[tmp["imp_sentence"]==1][1]))
    
    result_dict[sent] = answer

In [ ]:
sub_fi = pd.read_csv("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/extractive_sample_submission_v2.csv")
sub_fi["summary"] = sub_fi["id"].map(lambda x: result_dict[str(x)])
sub_fi.to_csv("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/ext_sub_nonorder.csv", index=False)

In [ ]:
sub_fi